# First steps in  *pandas*

***
> __Auteur__: Joseph Salmon
> <joseph.salmon@umontpellier.fr> , adapted from the notebook by Joris Van den Bossche:
https://github.com/jorisvandenbossche/pandas-tutorial/blob/master/01-pandas_introduction.ipynb

<a id="intro"> </a>

# Introduction et présentation

In [ ]:
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact  # widget manipulation

pd.options.display.max_rows = 8

## Cas 1: Survie sur le Titanic 

In [ ]:
from download import download

url = "http://josephsalmon.eu/enseignement/datasets/titanic.csv"
path_target = "./titanic.csv"
download(url, path_target, replace=False)

# df: data frame
df_titanic_raw = pd.read_csv("titanic.csv")

In [ ]:
df_titanic_raw.tail(n=3)

## Valeurs manquantes:
Pour faciliter la suite on ne garde que les observations qui sont complètes, on enlève donc ici les valeurs manquantes

In [ ]:
df_titanic = df_titanic_raw.dropna()
df_titanic.tail(3)

# Description succinte des variables:
- Survival - Survie (0 = Non; 1 = Oui).
- Pclass - Passenger Class / Classe du passager (1 = 1ere; 2 = 2nde; 3 = 3ème)
- Name - Nom
- Sex - Sexe
- Age - Age
- Sibsp - Nombre de frères / soeurs / maris / épouses à bord
- Parch - Nombre de parents ascendants / enfants à bord
- Ticket - Numéro du ticket
- Fare - Prix du ticket (British pound)
- Cabin - Cabine
- Embarked - Port d'embarquation (C = Cherbourg; Q = Queenstown; S = Southampton)

# Descriptif rapide:
- count - effectif
- mean - moyenne
- std (**st**andard **d**eviation - écart-type)


In [ ]:
df_titanic.describe()

## Compréhension visualisation de la base de données:

**Quelle est la répartition par âge des passagers?**

In [ ]:
plt.figure(figsize=(5, 5))
plt.hist(df_titanic['Age'], density=False, bins=50)
plt.xlabel('Age')
plt.ylabel('Proportion')
plt.title("Histogramme de l'âge des passagers")

In [ ]:
plt.figure(figsize=(5, 5))
# KDE: kernel density estimate
ax = sns.kdeplot(df_titanic['Age'], shade=True, cut=0, bw=5) # sns -> Seaborn package
plt.xlabel('Proportion')
plt.ylabel('Age')
ax.legend().set_visible(False)
plt.title("Estimation de la densité de l'âge des passagers")
plt.tight_layout()

## Widget
Interactive interaction with codes and output is nowdays easier and easier (see also Shiny app in R-software).
In `jupyter notebook` one can use for that `widgets` and the `interact` package. We are going to visualise that on the simple KDE and histograms examples.

In [ ]:
def hist_explore(n_bins=24):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.hist(df_titanic['Age'], density=True,
            bins=n_bins, alpha=0.25)  # standardization
    plt.xlabel('Age')
    plt.ylabel('Density level')
    plt.title("Histogram for passengers' age")
    plt.tight_layout()
    plt.show()

In [ ]:
interact(hist_explore, n_bins=(1, 30, 1))

In [ ]:
def kde_explore(bw=5):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax = sns.kdeplot(df_titanic['Age'], bw=bw, shade=True, cut=0)
    plt.xlabel('Age (in year)')
    plt.ylabel('Density level')
    plt.title("Age of the passengers")
    plt.tight_layout()
    plt.show()

In [ ]:
interact(kde_explore, bw=(0.001, 10, 0.1))

## `Groupby` function
How is the survival rate change w.r.t. to sex?

In [ ]:
df_titanic_raw.groupby('Sex')[['Survived']].aggregate(lambda x: x.mean())

How is the survival rate change w.r.t. the class?

In [ ]:
df_titanic.columns

In [ ]:
plt.figure()
df_titanic.groupby('Pclass')['Survived'].aggregate(lambda x: x.mean()).plot(kind='bar')

### <font color='red'> EXERCISE : median by class </font>
Add correct ylabel to the previous figure.

In [ ]:
# XXX to do

### <font color='red'> EXERCISE : median by class </font>
Perform a similar analysis with the median for the price per class in pounds.

## Catplot, or a visual groupby

In [ ]:
sns.catplot(x=df_titanic_raw.columns[2], y="Age",
            hue="Sex", data=df_titanic_raw, kind="violin", legend=False)
plt.title("Taux de survie par classe")
plt.tight_layout()

# Pandas: anlayser des données avec Python 

Pour les travaux intensifs en données en Python, la bibliothèque Pandas est devenue essentielle.

Qu'est ce que pandas? C'est un environnement qui gère des Data Frame:

- Pandas peut gérer *Data Frame* des tableaux *numpy* avec des étiquettes pour les lignes et les colonnes, et permet une prise en charge des types de données hétérogènes.
- Pandas peut aussi être considéré comme le data.frame de R en Python.
- Puissant pour travailler avec les données manquantes, travailler avec des données chronologiques, pour lire et écrire vos données, pour remodeler, regrouper, fusionner vos données, ...

Documentation: http://pandas.pydata.org/pandas-docs/stable/

Quand a-t-on besoin de Pandas?
Quand on travaille avec des tableaux ou des structures de données(commme des dataframe R, SQL table, Excel, Spreadsheet, ...):

- Importer des données
- Nettoyer des données "sales" 
- Explorer et comprendre des données
- Traiter et preparer les données pour faire une analyse 
- Analyser les données (avec en plus scikit-learn, statsmodels,...)
<br/>
<br/>

**ATTENTION / LIMITES:**

Pandas est bon pour travailler avec des données hétérogènes et des tableaux 1D/2D, mais tous les types de données ne correspondent pas à ces structures!

Contre-exemples:
- Quand on travaille avec des données de type **array** (e.g. images):  utiliser *numpy*
- Pour des données multidimensionnelles étiquetées  (e.g. données de climat): voir [xarray](http://xarray.pydata.org/en/stable/)

# Les structures de données en pandas : DataFrame et Series

Un DataFrame est une structure de données tabulaire (un objet multidimensionnel pouvant contenir des données étiquetées) composé de lignes et de colonnes, semblable à une feuille de calcul, une table de base de données ou à l'objet data.frame de R. Vous pouvez le considérer comme plusieurs objets Series partageant le même index.

In [ ]:
df_titanic

In [ ]:
df_titanic.index

In [ ]:
df_titanic.columns

In [ ]:
df_titanic.dtypes

In [ ]:
df_titanic.info()

In [ ]:
# Check that cabin is mostly missing, also the age
df_titanic_raw.info()

In [ ]:
array_titanic = df_titanic.values  # c'est la liste de valeur /array associé
array_titanic

### <font color='red'> EXERCISE : dropna</font>
Perform the following operation: remove the columns Cabin, and then remove the rows with missing age.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html


# Données uni-dimensionel : Series (une colonne d'un DataFrame)

Une Series est un support de base pour les données étiquetées unidimensionnelles.

In [ ]:
fare = df_titanic['Fare']

In [ ]:
fare

## Attributs de l'objet *Series*: indices et valeurs

In [ ]:
fare.values[:10]

In [ ]:
fare[6] # existe mais fare[0] provoque une erreur, car on l'a enlevé du dataFrame, comme valeur manquante.


Contrairement au tableau *numpy*, cet index peut être autre chose qu'un entier:

In [ ]:
df_titanic = df_titanic.set_index('Name')
df_titanic

In [ ]:
age = df_titanic['Age']
age['Behr, Mr. Karl Howell']

In [ ]:
age.mean()

In [ ]:
df_titanic[age <2]

In [ ]:
df_titanic_raw['Embarked'].value_counts()

In [ ]:
pd.options.display.max_rows = 70
df_titanic[df_titanic['Embarked']=='C']  # Les passagers montés à Cherbourg n'ont pas des noms gaulois...

In [ ]:
pd.options.display.max_rows = 8

In [ ]:
df_titanic_raw['Survived'].sum() / df_titanic_raw['Survived'].count()

In [ ]:
df_titanic['Survived'].mean()

** Quelle était la proportion de femmes sur le bateau? **

In [ ]:
df_titanic_raw.groupby(['Sex']).size() / df_titanic_raw['Sex'].count()

In [ ]:
df_titanic_raw.groupby(['Sex']).mean()

# Data import et export

Pandas supports nativement une large gamme de formats d'entrée / sortie:
- CSV, text
- SQL database
- Excel
- HDF5
- json
- html
- pickle
- sas, stata
- ...

In [ ]:
# pd.read_csv?

# Exploration

In [ ]:
df_titanic_raw.tail()

In [ ]:
df_titanic_raw.head()

In [ ]:
sns.set_palette("colorblind")
sns.catplot(x='Pclass',y='Age',hue='Survived',data=df_titanic_raw, kind="violin")

In [ ]:
df_titanic_raw.columns

# iloc

In [ ]:
df_titanic.iloc[0:2,1:8]

# loc

In [ ]:
df_titanic.loc['Bonnell, Miss. Elizabeth', 'Fare']

In [ ]:
df_titanic.loc['Bonnell, Miss. Elizabeth']

In [ ]:
df_titanic.loc['Bonnell, Miss. Elizabeth','Survived']= 100

In [ ]:
df_titanic.loc['Bonnell, Miss. Elizabeth']

In [ ]:
df_titanic.loc['Bonnell, Miss. Elizabeth','Survived']= 1  # On remet la valeur comme avant 

# group-by:

In [ ]:
df_titanic.groupby('Sex').mean()

In [ ]:
df_titanic_raw.groupby('Sex').mean()['Pclass']  # attention ici on prend toutes les données, meme les manquantes...

In [ ]:
df_titanic['AgeClass'] = pd.cut(df_titanic['Age'], bins=np.arange(0,90,10)) # créer des classes / découpes.

In [ ]:
df_titanic['AgeClass']

# Cas 2:  air quality in Paris.
(Source: Airparif)


In [ ]:
url = "http://josephsalmon.eu/enseignement/datasets/20080421_20160927-PA13_auto.csv"
path_target = "./20080421_20160927-PA13_auto.csv"
download(url, path_target, replace=False)

In [ ]:
!head -20 ./20080421_20160927-PA13_auto.csv

# Traitement des données temporelles et dates: 
https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

In [ ]:
polution_df = pd.read_csv('20080421_20160927-PA13_auto.csv', sep=';',
                          comment='#',
                          na_values="n/d",
                          converters={'heure': str})

In [ ]:
polution_df.head(12)

## Preprocess the data

### <font color='red'> EXERCISE : handling missing values </font>

What is the meaning of "na_values="n/d" above?

Note that an alternative can be obtained with the command `polution_df.replace('n/d', np.nan, inplace=True)`


In [ ]:
# check types
polution_df.dtypes

For more info on the object nature (inherited from numpy), see https://stackoverflow.com/questions/21018654/strings-in-a-dataframe-but-dtype-is-object

### First issue non conventional hours

In [ ]:
# start by changing to integer type (e.g. int8)
polution_df['heure'] = polution_df['heure'].astype(np.int8)

# no data is from 1 to 24... not conventional so let's make it from 0 to 23
polution_df['heure'] = polution_df['heure'] - 1

# and back to strings:
polution_df['heure'] = polution_df['heure'].astype('str')

### Time processing


In [ ]:
# https://www.tutorialspoint.com/python/time_strptime.htm

time_improved = pd.to_datetime(polution_df['date'] +
                               ' ' + polution_df['heure'] + ':00',
                               format='%d/%m/%Y %H:%M')

# Where d = day, m=month, Y=year, H=hour, M=minutes

In [ ]:
polution_df['date'] + ' ' + polution_df['heure'] + ':00'

In [ ]:
time_improved

In [ ]:
# create correct timing format in the dataframe
polution_df['DateTime'] = time_improved

# remove useles columns
del polution_df['heure']
del polution_df['date']

In [ ]:
# visualize the data set
polution_ts = polution_df.set_index(['DateTime'])
polution_ts = polution_ts.sort_index()
polution_ts.head(12)

In [ ]:
polution_ts.describe()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(6, 6), sharex=True)

axes[0].plot(polution_ts['O3'].resample('D').mean())
axes[0].set_title("Ozone polution: dayly average in Paris")
axes[0].set_ylabel("Concentration (µg/m³)")

axes[1].plot(polution_ts['NO2'].resample('D').mean())
axes[1].set_title("Nitrogen polution: dayly average in Paris")
axes[1].set_ylabel("Concentration (µg/m³)")

plt.show()

### <font color='red'> EXERCISE : worst of the day  </font>
Provide the same plots as before, but with dayly best and worst on the same figures (use different colors)

### Is the polution getting better along the year?

In [ ]:
ax = polution_ts['2008':].resample('A').mean().plot(figsize=(4,4))  # échantillone par année (A pour Annual)
plt.ylim(0,50)
plt.title("Evolution de la pollution: \n moyenne annuelle sur Paris")
plt.ylabel("Concentration (µg/m³)")
plt.xlabel("Années")

In [ ]:
# Chargement des couleurs
sns.set_palette("GnBu_d", n_colors=7)
polution_ts['weekday'] = polution_ts.index.weekday  # Monday=0, Sunday=6

# polution_ts['weekend'] = polution_ts['weekday'].isin([5, 6])

days = ['Lundi', 'Mardi', 'Mercredi',
        'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']

polution_week_no2 = polution_ts.groupby(['weekday', polution_ts.index.hour])[
    'NO2'].mean().unstack(level=0)
polution_week_03 = polution_ts.groupby(['weekday', polution_ts.index.hour])[
    'O3'].mean().unstack(level=0)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(7, 7), sharex=True)

polution_week_no2.plot(ax=axes[0])
axes[0].set_ylabel("Concentration (µg/m³)")
axes[0].set_xlabel("Heure de la journée")
axes[0].set_title(
    "Profil journalier de la pollution au NO2: effet du weekend?")
axes[0].set_xticks(np.arange(0, 24))
axes[0].set_xticklabels(np.arange(0, 24), rotation=45)
axes[0].set_ylim(0, 60)

polution_week_03.plot(ax=axes[1])
axes[1].set_ylabel("Concentration (µg/m³)")
axes[1].set_xlabel("Heure de la journée")
axes[1].set_title("Profil journalier de la pollution au O3: effet du weekend?")
axes[1].set_xticks(np.arange(0, 24))
axes[1].set_xticklabels(np.arange(0, 24), rotation=45)
axes[1].set_ylim(0, 70)
axes[0].legend().set_visible(False)
# ax.legend()
axes[1].legend(labels=days, loc='lower left', bbox_to_anchor=(1, 0.1))

plt.tight_layout()

In [ ]:
# XXX TODO quid des saisons?

In [ ]:
import calendar
polution_ts['month'] = polution_ts.index.month  # Janvier=0, .... Decembre=12
polution_ts['month'] = polution_ts['month'].apply(lambda x: calendar.month_abbr[x])
polution_ts.head()

In [ ]:
days = []

polution_month_no2 = polution_ts.groupby(['month', polution_ts.index.hour])[
    'NO2'].mean().unstack(level=0)
polution_month_03 = polution_ts.groupby(['month', polution_ts.index.hour])[
    'O3'].mean().unstack(level=0)

In [ ]:
sns.set_palette("GnBu_d", n_colors=12)

fig, axes = plt.subplots(2, 1, figsize=(7, 7), sharex=True)

polution_month_no2.plot(ax=axes[0])
axes[0].set_ylabel("Concentration (µg/m³)")
axes[0].set_xlabel("Heure de la journée")
axes[0].set_title(
    "Profil journalier de la pollution au NO2: effet du weekend?")
axes[0].set_xticks(np.arange(0, 24))
axes[0].set_xticklabels(np.arange(0, 24), rotation=45)
axes[0].set_ylim(0, 90)

polution_month_03.plot(ax=axes[1])
axes[1].set_ylabel("Concentration (µg/m³)")
axes[1].set_xlabel("Heure de la journée")
axes[1].set_title("Profil journalier de la pollution au O3: effet du weekend?")
axes[1].set_xticks(np.arange(0, 24))
axes[1].set_xticklabels(np.arange(0, 24), rotation=45)
axes[1].set_ylim(0, 90)
axes[0].legend().set_visible(False)
# ax.legend()
axes[1].legend(labels=calendar.month_name[1:], loc='lower left', bbox_to_anchor=(1, 0.1))

plt.tight_layout()

# Your turn: explore the bike acceident dataset

https://www.data.gouv.fr/fr/datasets/accidents-de-velo-en-france/

Possible visualisation
https://koumoul.com/en/datasets/accidents-velos

In [ ]:
url = "https://www.data.gouv.fr/fr/datasets/r/ab84353b-498b-4ef5-9a02-a6403f2ead96"
path_target = "./bicycle_db.csv"
download(url, path_target, replace=False)

In [ ]:
# df: data frame
df_bikes = pd.read_csv("bicycle_db.csv",na_values="", converters={'data': str, 'heure': str})

In [ ]:
!head -5 ./bicycle_db.csv

In [ ]:
df_bikes.head()

In [ ]:
df_bikes['existence securite'].unique()

In [ ]:
df_bikes['gravite accident'].unique()

### Handle missing values in `heure`

In [ ]:
df_bikes['date'].hasnans

In [ ]:
df_bikes['heure'].hasnans

In [ ]:
pd.options.display.max_rows = 20
df_bikes.iloc[400:402]

In [ ]:
# remove missing hours cases by np.nan
df_bikes['heure']=df_bikes['heure'].replace('', np.nan)
df_bikes.iloc[400:402]

In [ ]:
df_bikes.dropna(subset=['heure'], inplace=True)
df_bikes.iloc[399:402]

### <font color='red'> EXERCISE : Dates?  </font>
Can you find the starting day and the ending day of the study automatically? 
hint sort the data.
You can sort the data by time, , say with df.sort('Time') )

In [ ]:
df_bikes['date'] + ' ' + df_bikes['heure'] + ':00'

In [ ]:
# ADAPT OLD to create the df_bikes['Time']

# time_improved = pd.to_datetime(df_bikes['date'] +
#                                ' ' + df_bikes['heure'] + ':00',
#                                format='%d/%m/%Y %H:%M')

# Where d = day, m=month, Y=year, H=hour, M=minutes
# create correct timing format in the dataframe


In [ ]:
df_bikes['Time'] = time_improved

# remove useles columns
del polution_df['heure']
del polution_df['date']

### <font color='red'> EXERCISE : Is the helmet saving your life?  </font>
Peform an analysis so that you can check the benefit or not of wearing helmet to save your life.  

### <font color='red'> EXERCISE : Are men and women dying equally on a bike?  </font>
Peform an analysis to check any difference between men and woman survival on a bike?

### <font color='red'> EXERCISE : Accident during the week?  </font>
Peform an analysis to check when the accidents are occuring during the week.